# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [31]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [32]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [33]:
%store -r numerosKalaba
numerosKalaba=[1,2,3,4,5]
numerosKalaba=[5]
print numerosKalaba
%store numerosKalaba

[5]
Stored 'numerosKalaba' (list)


In [34]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
serie=repertoire+"21-"
nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]

In [35]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [36]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    lexiqueTestPrep=getExemple(stems["PREP"])
    lexiqueTestNoun=getExemple(stems["NOM"])
    lexiqueTestHyper=getExemple(stems["ADJ"])
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/21-K5/
NOM : Genre Nombre Cas
VER : Temps Pers Genre
ADJ : Genre Nombre Cas
PRO : Genre Nombre Cas
DET : Genre Nombre Cas
head stems
head stems,NOM
head stems,NOM,N1
head stems,NOM,N2
villageoise villageois villageoise
villageoises villageois villageoise
lumière lumière lumière
lumières lumière lumière
dernIER dernier-N dernier-NOM
dernIÈRE dernier-N dernier-NOM
dernIERs dernier-N dernier-NOM
dernIÈREs dernier-N dernier-NOM
head stems,NOM,N3
demande demande demande
demandes demande demande
soeur soeur soeur
soeurs soeur soeur
head stems,NOM,N4
nouvELLE nouvelle-NOM nouvelle-N
nouvELLEs nouvelle-NOM nouvelle-N
head stems,VER
head stems,VER,VI
head stems,VER,VT
nourrit nourrir nourrir
excuse excuser excuse
head stems,ADJ

=======ERREURS========

Formes homographes "villageoise" pour "villageois" et "villageoise"
Formes homographes "villageoises" pour "villageois" et "villageoise"
Formes homographes "lumière" p

In [37]:
getExemple(stems["PREP"])

'par'

In [38]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

trois.A1
table.N1.A
par


In [39]:
paradigmes.getSigmas(["VER"])

[u'VER, Temps=PRS, Pers=3Sg, Genre=A',
 u'VER, Temps=PRS, Pers=3Sg, Genre=B',
 u'VER, Temps=PRS, Pers=3Sg, Genre=C',
 u'VER, Temps=PRS, Pers=3Sg, Genre=D',
 u'VER, Temps=PRS, Pers=3Pau, Genre=A',
 u'VER, Temps=PRS, Pers=3Pau, Genre=B',
 u'VER, Temps=PRS, Pers=3Pau, Genre=C',
 u'VER, Temps=PRS, Pers=3Pau, Genre=D',
 u'VER, Temps=PRS, Pers=3Pl, Genre=A',
 u'VER, Temps=PRS, Pers=3Pl, Genre=B',
 u'VER, Temps=PRS, Pers=3Pl, Genre=C',
 u'VER, Temps=PRS, Pers=3Pl, Genre=D',
 u'VER, Temps=PST, Pers=3Sg, Genre=A',
 u'VER, Temps=PST, Pers=3Sg, Genre=B',
 u'VER, Temps=PST, Pers=3Sg, Genre=C',
 u'VER, Temps=PST, Pers=3Sg, Genre=D',
 u'VER, Temps=PST, Pers=3Pau, Genre=A',
 u'VER, Temps=PST, Pers=3Pau, Genre=B',
 u'VER, Temps=PST, Pers=3Pau, Genre=C',
 u'VER, Temps=PST, Pers=3Pau, Genre=D',
 u'VER, Temps=PST, Pers=3Pl, Genre=A',
 u'VER, Temps=PST, Pers=3Pl, Genre=B',
 u'VER, Temps=PST, Pers=3Pl, Genre=C',
 u'VER, Temps=PST, Pers=3Pl, Genre=D']

In [40]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [41]:
regles.getRules("VER",'CF=V1, Temps=PRS, Pers=3PL')
# regles.getRules("PRO","Nombre=Du, Genre=C")

[('1A22e3', 'Temps=PRS')]

In [42]:
test=Lexeme("ganav","VER","croc").getParadigm(lignes=["Genre"],colonne=["Pers","Temps"])
test=Lexeme("ganav","NOM","croc").getParadigm(lignes=["Nombre"],colonne=["Cas"])

# test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre","Cas"])
test
#test.index.sortlevel(0,ascending=False)

del Genre


Cas,Abs,Dat,Erg,Obl
Nombre,,,,
Pau,ganav,ganav,ganav,ganav
Pl,ganav,ganav,ganav,ganav
Sg,ganav,ganav,ganav,ganav


In [43]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{u'A': {('DeTap', u'caf\xe9'),
  ('DoDal', u'p\xe8re'),
  ('Dodor', u'lumi\xe8re'),
  ('DuguN', u'for\xeat'),
  ('Nipet', u'bras'),
  ('damak', u'chambre'),
  ('dimuN', u'homme'),
  ('dobin', u'fille'),
  ('dogab', u'louve'),
  ('gufeS', u'col\xe8re'),
  ('katiSa', u'Katisha'),
  ('kerib', u'blessure'),
  ('kides', u'soeur'),
  ('maguS', u'obscurit\xe9'),
  ('makert', u'sorcier'),
  ('mejaN', u'h\xe9ros'),
  ('mikot', u'marais'),
  ('moger', u'tornade'),
  ('mogul', u'histoire'),
  ('mowat', u'milieu'),
  ('muSun', u'combattant'),
  ('nanir', u'villageoise'),
  ('nubap', u'coyote'),
  ('pikag', u'chemin'),
  ('pomaD', u'ma\xeetre'),
  ('sariv', u'couteau'),
  ('sovuT', u'ombre'),
  ('tarem', u'm\xe8re'),
  ('teNak', u'table'),
  ('vjolet', u'Violette'),
  ('wubiD', u'nouvelle-nom'),
  ('ziden', u'gorge')},
 u'B': {('Dodor', u'lumi\xe8re'),
  ('Nubif', u'esprit'),
  ('Sebep', u'place'),
  ('Soniv', u'mort'),
  ('TaDaz', u'cri'),
  ('TaDod', u'\u0153uf'),
  ('Tejiv', u'corps'),
  ('Tobaz

In [44]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

fuir.VT.V1 Zetak
chasser.VT.V2 Zuruk
entrer.VI.V1 wajut
feu.N4.B kobud
pendre.VT.V1 Duwif
trois.A1 poker
Clemencia.N3.C klemansja
villageoise.N2.A nanir
planter.VT.V2 SukoD
apporter.VT.V1 goben
retourner.VI.V2 bagut
reconstruire.VT.V2 posaf
oeil.N1.C revover
créature.N1.B Tobaz
dernier-n.N2.D ruwoN
chef.N1.C porir
découvrir.VT.V2 TuteN
échanger.VT.V2 siduT
nourrir.VT.V1 mutase
furieux.A1 togap
Hoderi.N2.B oderi
rejoindre.VT.V2 riluS
placer.VT.V1 makuS
changer.VT.V2 tedut
différent.A2 valir
construction.N2.D gotuz
enfermer.VT.V1 fetek
viande.N3.D gakil
protéger.VT.V1 muTib
courir.VI.V1 tidag
excuse.N4.D magok
proximité.N3.D serkam
père.N4.A DoDal
envahir.VT.V2 varak
Violette.N3.A vjolet
lueur.N4.C Sibur
dans feD
effrayé.A1 malaz
Nicole.N1.C nikol
deux.A1 loNak
surplomber.VT.V1 toNet
troisième.A2 okarina
bûcher.N3.B pabov
connaître.VT.V1 Degem
Elphaba.N2.D elfaba
demander.VT.V1 furun
demande.N1.D tinup
cinq.A1 kink
devant lolon
sort.N3.C gijit
Nabil.N4.D Nabil
raison.N4.C favet
méchant.A

In [45]:
ding()